# EDA

In [1]:
import pandas as pd
df = pd.read_excel('/content/jeans_data_full_dataset.xlsx')


In [2]:
df

,week,price,sales,special_event,se_indicator
0,1,94,259,New Year's Day,1
1,2,135,69,NaN,0
2,3,135,59,NaN,0
3,4,81,149,NaN,0
4,5,135,72,NaN,0
...,...,...,...,...,...
515,516,108,222,Black Friday,1
516,517,135,56,NaN,0
517,518,135,56,NaN,0
518,519,74,205,Christmas Eve,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   week           520 non-null    int64 
 1   price          520 non-null    int64 
 2   sales          520 non-null    int64 
 3   special_event  120 non-null    object
 4   se_indicator   520 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 20.4+ KB


# Model I

In [5]:
import statsmodels.api as sm
X = sm.add_constant(df[["se_indicator","price"]])

# fit the OLS model
olsjeans = sm.OLS(df['sales'], X).fit()

# results summary
olsjeans.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     3427.
Date:                Mon, 25 Mar 2024   Prob (F-statistic):          4.66e-299
Time:                        23:10:38   Log-Likelihood:                -2253.1
No. Observations:                 520   AIC:                             4512.
Df Residuals:                     517   BIC:                             4525.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          252.0725      7.902     31.901      0.000     236.549     267.596
se_indicator   100.9494      3.019     33.440      0.000      95.019     106.880
price           -1.3523      0.060    -22.494      0.000      -1.470      -1.234
==============================================================================
Omnibus:                       52.143   Durbin-Watson:                   2.064
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              139.683
Skew:                           0.493   Prob(JB):                     4.66e-31
Kurtosis:                       5.340   Cond. No.                     1.26e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
import statsmodels.api as sm

# Define the the price and se_indicator
price = 110
se_indicator = 1

# Predict total sales
predicted_total_sales = olsjeans.predict([1,se_indicator, price])
predicted_total_sales= round(predicted_total_sales[0])
# Print the predicted total sales
print("Predicted Total Sales of modle I:", predicted_total_sales)


Predicted Total Sales of modle I: 204


From the model I's regression results,the **baseline** (the Constant)is 252, which means that all explanatory variables have a zero, and epected level of sales would be 252.


**Price**: The negative coefficient of -1.35 means a negative correlation between sales and prices, if all other varaible holds constant, 1 dollar price increase, the sales would decrease in 1.35 unit.

**Special even**t: The cofficient of 100.95 suggest a postive correlation between sales and Special event, if all other varaible holds constant, if special event is happening , the sales would increase by 100.94 unit.

All cofficients are statstically significant as p values are smaller than 0.05.

**R^2**: The porportion of variance of depedent variable that is explained by independent variables is 93%.

**Epected Sales**: if the price is set at 110, which is 135 -25 dollars of discount and speical event is happening, then the epected sales would be 204 pairs.


# Model II

In [7]:
#calculate a discount_percentage
df['discount_percentage'] = ((135 - df['price']) / 135) * 100

# Define independent variables (se and discount percentage) and dependent variable (target)
X_model2 = sm.add_constant(df[["se_indicator", "discount_percentage"]])
y_model2 = df['sales']

ols_model2 = sm.OLS(y_model2, X_model2).fit()

# Results for Model II
print("\nModel II Summary:")
print(ols_model2.summary())


Model II Summary:
                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.930
Method:                 Least Squares   F-statistic:                     3427.
Date:                Mon, 25 Mar 2024   Prob (F-statistic):          4.66e-299
Time:                        23:10:38   Log-Likelihood:                -2253.1
No. Observations:                 520   AIC:                             4512.
Df Residuals:                     517   BIC:                             4525.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const          

In [8]:
df

,week,price,sales,special_event,se_indicator,discount_percentage
0,1,94,259,New Year's Day,1,30.370370
1,2,135,69,NaN,0,0.000000
2,3,135,59,NaN,0,0.000000
3,4,81,149,NaN,0,40.000000
4,5,135,72,NaN,0,0.000000
...,...,...,...,...,...,...
515,516,108,222,Black Friday,1,20.000000
516,517,135,56,NaN,0,0.000000
517,518,135,56,NaN,0,0.000000
518,519,74,205,Christmas Eve,1,45.185185


In [9]:
#define discount percentage_promo
discount_percentage_promo = (135-110 )/ 135 * 100
discount_percentage_promo

# Use ols_model2 Model to predict sales for the "$25 Off" promotion with percentage
predicted_sales_promo = ols_model2.predict([1, se_indicator, discount_percentage_promo])
predicted_sales_promo=round(predicted_sales_promo[0])
print("Predicted Total Sales of modle II:", predicted_sales_promo)


Predicted Total Sales of modle II: 204


For the Model II, the difference is that I used discount_percentage as independt variable instead of price. Even though indepent variable is different, I still got the same predicted sales of 204 pairs of jeans.

From the model II's regression results,the **baseline** (the Constant) is 69.5, which means that all explanatory variables have a zero, and epected level of sales would be 69.5.


**Discount Percentage**: The positive coefficient of 1.83 means a negative correlation between sales and prices, if all other varaible holds constant, 1 dollar price increase, the sales would increase in 1.83 unit.

**Special event**: The cofficient of 100.95 suggest a postive correlation between sales and Special event, if all other varaible holds constant, if special event is happening , the sales would increase by 100.94 unit.

All cofficients are statstically significant as p values are smaller than 0.05.

**R^2**: The porportion of variance of depedent variable that is explained by independent variables is 93%.

**Epected Sales**: if the price is set at 110, which is 135 -25 dollars of discount and speical event is happening, then the epected sales would be 204 pairs.
